## Loading Libraries and Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
pd.set_option('display.max_columns', None)
%matplotlib inline
raw_data=pd.read_csv('Rincon3601.csv',low_memory=False,encoding='utf-8')
raw_data.shape

(98893, 174)

In [2]:
df=raw_data.copy()
df=df.dropna(axis=1, how='all')
df=df.dropna(axis=0, how='all')
df=df.dropna(subset=['PercentSafe'])
df.shape

(17029, 170)

droping columns with missing values more then 80%

In [3]:
df=df[df.columns[df.isnull().mean() < 0.5]]
df.shape

(17029, 82)

### remove correlated columns

In [4]:
df1=df.copy()

In [5]:
def find_correlation(data, threshold=0.75):
    corr_mat = data.corr()
    corr_mat.loc[:, :] = np.tril(corr_mat, k=-1)
    already_in = set()
    result = []
    for col in corr_mat:
        perfect_corr = corr_mat[col][corr_mat[col] > threshold].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)
    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

In [6]:
corr_col=find_correlation(df1)
corr_col

['CompanyGroupID',
 'CleanSheets_n',
 'NumberObserved',
 'Sheet_n',
 'PPE',
 'Procedures',
 'Tools_and_Equipment',
 'Work_Environment',
 'BodyDyn_PinchPtRedZoneLineFire',
 'PPE_HandArm',
 'Procedures_BarricadeWarning',
 'ToolsEquip_ConditionSelectionUse',
 'WorkEnvironment_WalkWorkSurface',
 'SumRisks',
 'TimeofDay_Day',
 'MOP_BAPP']

In [7]:
df1=df1.drop(columns=corr_col,axis=1)
df1.shape

(17029, 66)

### remove high cardinality

In [8]:
df2=df1.copy()
df2.columns

Index(['ProjectNo', 'CompanyNameID', 'ProjectDesc_scrub', 'City',
       'CountryRegion', 'Country', 'RD_WorldRegion', 'Type',
       'InitialConsultant', 'ConsType', 'HWC', 'BAPPLicensed',
       'Earliest_Intervention', 'BAPPImpWhat', 'BAPPImpHow', 'Focus', 'DOB',
       'Acronym', 'CRMMarketSegment', 'RD_Industry', 'NotesScrubbed',
       'Include_YN', 'FOM', 'Observers_n', 'SumSafes', 'Percent_CleanSheets',
       'ExposuresObserved_n', 'Body_Dynamics', 'MotorVeh_Equip',
       'BodyDyn_AscDesc', 'BodyDyn_EyesTaskHand', 'BodyDyn_EyesPathRoad',
       'BodyDyn_LiftLowerTwist', 'PPE_Body', 'PPE_FaceHeadHearing',
       'PPE_FootLegKnee', 'PPE_Hearing', 'PPE_Respiratory',
       'Procedures_EnergyIsolation', 'Procedures_WorkingatHeights',
       'WorkEnvironment_HousekpingStorg', 'WorkGroup_AdminOffice',
       'Workgroup_DistributionWarehouse', 'Workgroup_LabRD',
       'Workgroup_MaintenanceShopMech', 'ObservationType_Coached',
       'EmployeeType_Contactor', 'HoursintoShift_Overti

In [1]:
# df2.Acronym.value_counts()

In [10]:
df2.head()

,ProjectNo,CompanyNameID,ProjectDesc_scrub,City,CountryRegion,Country,RD_WorldRegion,Type,InitialConsultant,ConsType,HWC,BAPPLicensed,Earliest_Intervention,BAPPImpWhat,BAPPImpHow,Focus,DOB,Acronym,CRMMarketSegment,RD_Industry,NotesScrubbed,Include_YN,FOM,Observers_n,SumSafes,Percent_CleanSheets,ExposuresObserved_n,Body_Dynamics,MotorVeh_Equip,BodyDyn_AscDesc,BodyDyn_EyesTaskHand,BodyDyn_EyesPathRoad,BodyDyn_LiftLowerTwist,PPE_Body,PPE_FaceHeadHearing,PPE_FootLegKnee,PPE_Hearing,PPE_Respiratory,Procedures_EnergyIsolation,Procedures_WorkingatHeights,WorkEnvironment_HousekpingStorg,WorkGroup_AdminOffice,Workgroup_DistributionWarehouse,Workgroup_LabRD,Workgroup_MaintenanceShopMech,ObservationType_Coached,EmployeeType_Contactor,HoursintoShift_Overtime,ObservationType_Self,ObservationType_SpecialFocus,TimeofDay_Evening,TimeofDay_Morning,TimeofDay_Night,WeatherTemp_Cold,WeatherTemp_Hot,WeatherTemp_Mild,WeatherTemp_Dry,WeatherTemp_SnowFrostSleet,WeatherTemp_Wet,Workstate_UpsetEmergency,Workstate_PlannedMaintenance,PrePost,PrePostEXP,PercentRisk,PercentSafe,ProjectNoID
334,ACSDR000,146,NaN,Drayton,ND,US,USA,BST Implementation,Jacque Cooney,BST,250.0,BAPP User; OCDI,01jun2003 00:00:00,All4,TrainT,Safety,01oct2003 00:00:00,NaN,Food Manufacturing,Food,5/2004-gcm: Per JSC client consideres 11/03 th...,1,01oct2003 00:00:00,8.0,55.0,0.1250,27.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,1.0,1.0,0.0,0.0,1.0,1.0,21.428571,78.571429,ACSDR000
335,ACSDR000,146,NaN,Drayton,ND,US,USA,BST Implementation,Jacque Cooney,BST,250.0,BAPP User; OCDI,01jun2003 00:00:00,All4,TrainT,Safety,01oct2003 00:00:00,NaN,Food Manufacturing,Food,5/2004-gcm: Per JSC client consideres 11/03 th...,1,01nov2003 00:00:00,9.0,193.0,0.4800,27.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,3.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,5.0,14.0,11.0,0.0,2.0,0.0,1.0,1.0,2.0,8.095238,91.904762,ACSDR000
336,ACSDR000,146,NaN,Drayton,ND,US,USA,BST Implementation,Jacque Cooney,BST,250.0,BAPP User; OCDI,01jun2003 00:00:00,All4,TrainT,Safety,01oct2003 00:00:00,NaN,Food Manufacturing,Food,5/2004-gcm: Per JSC client consideres 11/03 th...,1,01dec2003 00:00:00,24.0,700.0,0.3623,27.0,20.0,1.0,0.0,0.0,1.0,6.0,1.0,7.0,2.0,8.0,1.0,2.0,NaN,7.0,0.0,9.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,0.0,17.0,41.0,37.0,1.0,7.0,0.0,4.0,1.0,3.0,8.972692,91.027308,ACSDR000
337,ACSDR000,146,NaN,Drayton,ND,US,USA,BST Implementation,Jacque Cooney,BST,250.0,BAPP User; OCDI,01jun2003 00:00:00,All4,TrainT,Safety,01oct2003 00:00:00,NaN,Food Manufacturing,Food,5/2004-gcm: Per JSC client consideres 11/03 th...,1,01jan2004 00:00:00,17.0,766.0,0.3699,27.0,14.0,0.0,2.0,0.0,0.0,5.0,3.0,1.0,1.0,3.0,0.0,4.0,NaN,4.0,0.0,6.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,10.0,0.0,8.0,45.0,29.0,0.0,5.0,0.0,4.0,1.0,4.0,11.239861,88.760139,ACSDR000
338,ACSDR000,146,NaN,Drayton,ND,US,USA,BST Implementation,Jacque Cooney,BST,250.0,BAPP User; OCDI,01jun2003 00:00:00,All4,TrainT,Safety,01oct2003 00:00:00,NaN,Food Manufacturing,Food,5/2004-gcm: Per JSC client consideres 11/03 th...,1,01feb2004 00:00:00,9.0,335.0,0.2632,27.0,20.0,0.0,3.0,1.0,1.0,7.0,2.0,0.0,0.0,0.0,2.0,0.0,NaN,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,4.0,18.0,14.0,6.0,3.0,0.0,4.0,1.0,5.0,18.092910,81.907090,ACSDR000


In [11]:
remove_high_cardinality=['Acronym','City','CountryRegion','DOB','Earliest_Intervention','FOM','InitialConsultant',
                         'NotesScrubbed','ObservationType_Self','Percent_CleanSheets','Procedures_EnergyIsolation',
                        'ProjectDesc_scrub','ProjectNo','ProjectNoID','PercentRisk','CompanyNameID']

In [12]:
df2=df2.drop(columns=remove_high_cardinality,axis=1)
df2.shape

(17029, 50)

### handling missing value

In [13]:
bnp_df=df2.copy()
for f in bnp_df.columns:
    # fill NaN values with mean
    if bnp_df[f].dtype == 'float64':
        bnp_df[f][np.isnan(bnp_df[f])] = bnp_df[f].mean()
        
        
    # fill NaN values with most occured value
    elif bnp_df[f].dtype == 'object':
        bnp_df[f][bnp_df[f] != bnp_df[f]] = bnp_df[f].value_counts().index[0]

C:\ProgramData\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [14]:
bnp_df.head()

,Country,RD_WorldRegion,Type,ConsType,HWC,BAPPLicensed,BAPPImpWhat,BAPPImpHow,Focus,CRMMarketSegment,RD_Industry,Include_YN,Observers_n,SumSafes,ExposuresObserved_n,Body_Dynamics,MotorVeh_Equip,BodyDyn_AscDesc,BodyDyn_EyesTaskHand,BodyDyn_EyesPathRoad,BodyDyn_LiftLowerTwist,PPE_Body,PPE_FaceHeadHearing,PPE_FootLegKnee,PPE_Hearing,PPE_Respiratory,Procedures_WorkingatHeights,WorkEnvironment_HousekpingStorg,WorkGroup_AdminOffice,Workgroup_DistributionWarehouse,Workgroup_LabRD,Workgroup_MaintenanceShopMech,ObservationType_Coached,EmployeeType_Contactor,HoursintoShift_Overtime,ObservationType_SpecialFocus,TimeofDay_Evening,TimeofDay_Morning,TimeofDay_Night,WeatherTemp_Cold,WeatherTemp_Hot,WeatherTemp_Mild,WeatherTemp_Dry,WeatherTemp_SnowFrostSleet,WeatherTemp_Wet,Workstate_UpsetEmergency,Workstate_PlannedMaintenance,PrePost,PrePostEXP,PercentSafe
334,US,USA,BST Implementation,BST,250.0,BAPP User; OCDI,All4,TrainT,Safety,Food Manufacturing,Food,1,8.0,55.0,27.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.895764,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,1.0,1.0,0.0,0.0,1.0,1.0,78.571429
335,US,USA,BST Implementation,BST,250.0,BAPP User; OCDI,All4,TrainT,Safety,Food Manufacturing,Food,1,9.0,193.0,27.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.895764,3.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,5.0,14.0,11.0,0.0,2.0,0.0,1.0,1.0,2.0,91.904762
336,US,USA,BST Implementation,BST,250.0,BAPP User; OCDI,All4,TrainT,Safety,Food Manufacturing,Food,1,24.0,700.0,27.0,20.0,1.0,0.0,0.0,1.0,6.0,1.0,7.0,2.0,8.0,1.0,1.895764,7.0,0.0,9.0,3.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,0.0,17.0,41.0,37.0,1.0,7.0,0.0,4.0,1.0,3.0,91.027308
337,US,USA,BST Implementation,BST,250.0,BAPP User; OCDI,All4,TrainT,Safety,Food Manufacturing,Food,1,17.0,766.0,27.0,14.0,0.0,2.0,0.0,0.0,5.0,3.0,1.0,1.0,3.0,0.0,1.895764,4.0,0.0,6.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,10.0,0.0,8.0,45.0,29.0,0.0,5.0,0.0,4.0,1.0,4.0,88.760139
338,US,USA,BST Implementation,BST,250.0,BAPP User; OCDI,All4,TrainT,Safety,Food Manufacturing,Food,1,9.0,335.0,27.0,20.0,0.0,3.0,1.0,1.0,7.0,2.0,0.0,0.0,0.0,2.0,1.895764,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,4.0,18.0,14.0,6.0,3.0,0.0,4.0,1.0,5.0,81.907090


In [15]:
#bnp_df.to_csv('dekra_clean1.csv',index=False)

## Method1

In [16]:
df3=bnp_df.copy()
df3.shape

(17029, 50)

In [17]:
from sklearn import preprocessing
for f in df3.columns:
    if df3[f].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.unique(list(df3[f].values)))
        df3[f]   = lbl.transform(list(df3[f].values))

In [18]:
df3.head()

,Country,RD_WorldRegion,Type,ConsType,HWC,BAPPLicensed,BAPPImpWhat,BAPPImpHow,Focus,CRMMarketSegment,RD_Industry,Include_YN,Observers_n,SumSafes,ExposuresObserved_n,Body_Dynamics,MotorVeh_Equip,BodyDyn_AscDesc,BodyDyn_EyesTaskHand,BodyDyn_EyesPathRoad,BodyDyn_LiftLowerTwist,PPE_Body,PPE_FaceHeadHearing,PPE_FootLegKnee,PPE_Hearing,PPE_Respiratory,Procedures_WorkingatHeights,WorkEnvironment_HousekpingStorg,WorkGroup_AdminOffice,Workgroup_DistributionWarehouse,Workgroup_LabRD,Workgroup_MaintenanceShopMech,ObservationType_Coached,EmployeeType_Contactor,HoursintoShift_Overtime,ObservationType_SpecialFocus,TimeofDay_Evening,TimeofDay_Morning,TimeofDay_Night,WeatherTemp_Cold,WeatherTemp_Hot,WeatherTemp_Mild,WeatherTemp_Dry,WeatherTemp_SnowFrostSleet,WeatherTemp_Wet,Workstate_UpsetEmergency,Workstate_PlannedMaintenance,PrePost,PrePostEXP,PercentSafe
334,38,7,2,1,250.0,5,0,4,0,4,5,18,8.0,55.0,27.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.895764,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,1.0,1.0,0.0,0.0,1.0,1.0,78.571429
335,38,7,2,1,250.0,5,0,4,0,4,5,18,9.0,193.0,27.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.895764,3.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,5.0,14.0,11.0,0.0,2.0,0.0,1.0,1.0,2.0,91.904762
336,38,7,2,1,250.0,5,0,4,0,4,5,18,24.0,700.0,27.0,20.0,1.0,0.0,0.0,1.0,6.0,1.0,7.0,2.0,8.0,1.0,1.895764,7.0,0.0,9.0,3.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,0.0,17.0,41.0,37.0,1.0,7.0,0.0,4.0,1.0,3.0,91.027308
337,38,7,2,1,250.0,5,0,4,0,4,5,18,17.0,766.0,27.0,14.0,0.0,2.0,0.0,0.0,5.0,3.0,1.0,1.0,3.0,0.0,1.895764,4.0,0.0,6.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,10.0,0.0,8.0,45.0,29.0,0.0,5.0,0.0,4.0,1.0,4.0,88.760139
338,38,7,2,1,250.0,5,0,4,0,4,5,18,9.0,335.0,27.0,20.0,0.0,3.0,1.0,1.0,7.0,2.0,0.0,0.0,0.0,2.0,1.895764,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,4.0,18.0,14.0,6.0,3.0,0.0,4.0,1.0,5.0,81.907090


In [19]:
#df3.to_csv('dekra_clean2.csv',index=False)

In [ ]:
X = df3.drop(["PercentSafe"],axis=1)
Y = df3["PercentSafe"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=0)
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test,y_pred))

## Method 2

In [ ]:
df4=bnp_df.copy()
df4.shape

In [ ]:
col_to_trans=['Country','RD_WorldRegion','Type','ConsType','BAPPLicensed','BAPPImpWhat','BAPPImpHow','Focus'
              ,'CRMMarketSegment','RD_Industry','Include_YN',]
df4=pd.get_dummies(df4,columns=col_to_trans)
df4.shape

In [ ]:
X = df4.drop(["PercentSafe"],axis=1)
Y = df4["PercentSafe"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=0)
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test,y_pred))

## Method 3

In [ ]:
df=raw_data.copy()
df=df.dropna(axis=1, how='all')
df=df.dropna(axis=0, how='all')
df=df.dropna(subset=['PercentSafe'])
df.shape

In [ ]:
df5=df.copy()

In [ ]:
corr_col=find_correlation(df5)
corr_col

In [ ]:
df5=df5.drop(columns=corr_col,axis=1)
df5.shape

In [ ]:
df5=df5[df5.columns[df5.isnull().mean() < 0.8]]
df5.shape

In [ ]:
bnp_df=df5.copy()
for f in bnp_df.columns:
    # fill NaN values with mean
    if bnp_df[f].dtype == 'float64':
        bnp_df[f][np.isnan(bnp_df[f])] = bnp_df[f].mean()
        
        
    # fill NaN values with most occured value
    elif bnp_df[f].dtype == 'object':
        bnp_df[f][bnp_df[f] != bnp_df[f]] = bnp_df[f].value_counts().index[0]

In [ ]:
from sklearn import preprocessing

for f in bnp_df.columns:
    if bnp_df[f].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.unique(list(bnp_df[f].values)))
        bnp_df[f]   = lbl.transform(list(bnp_df[f].values))

In [ ]:
X = bnp_df.drop(["PercentSafe"],axis=1)
Y = bnp_df["PercentSafe"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=0)
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test,y_pred))